# Running a 5-Point Calibration Scan With OSO

This notebook adapts the Five Point Calibration Scan notebook to work with PSI. Running this notebook requires that a PSI namespace be spun up with `DISH_LMC_ENABLED` set to true.

This notebook can be Run with BITE or SPFRx, with BITE requiring the optional BITE notebook to be run. 

## 1 Setup 

### 1.1 Environment Setup

In [ ]:
import sys

sys.path.append("../../src")

import json
import os
from time import sleep

from ska_oso_pdm import SBDefinition
from ska_oso_scripting import oda_helper
from ska_oso_scripting.functions import pdm_transforms
from ska_oso_scripting.functions.sb import create_sbi, load_sbd
from ska_oso_scripting.objects import SubArray, Telescope
from tango import DevFailed, DeviceProxy

import notebook_tools.wait_for_tango as wait_for_tango

### 1.2 Set Variables

First, grab the namespace launched from the pipeline:

In [ ]:
!kubectl get ns | grep ska-mid-psi 

And load it into the variables for the notebook:

In [ ]:
namespace = "ci-ska-mid-psi-1386660462-alexschell"  # set to desired NS
simulation_mode = 0  # set to 1 to run in sim mode
target_boards_list = [1]  # assign boards
test_id = "talon2 basic gaussian noise"  # Test to send config from
server = "ska-sdp-kafka." + namespace + ".svc.cluster.local:9092"
subarray_id = 1

Get the SKUID pod:

In [ ]:
!kubectl get pods -n $namespace | grep skuid

And set the id value based on the name of this pod:

In [ ]:
skuid_id = "ska-ser-skuid-test-1386660462-svc"
kafka_id = ""

Next, load all the other vars the notebook will use. These should not need to be changed for this run.

In [ ]:
# Fully Qualified Domain Names for the devices to set up proxies
CSP_CONTROLLER_FQDN = "mid-csp/control/0"
CSP_SUBARRAY_FQDN = "mid-csp/subarray/01"

CENTRAL_NODE_FQDN = "ska_mid/tm_central/central_node"
LEAF_NODE_SUBARRAY_FQDN = "ska_mid/tm_leaf_node/csp_subarray01"
TMC_SUBARRAY_FQDN = "ska_mid/tm_subarray_node/1"

LEAF_NODE_MASTER_FQDN = "ska_mid/tm_leaf_node/csp_master"

CBF_SUBARRAY_FQDN = "mid_csp_cbf/sub_elt/subarray_01"

BITE_FQDN = "mid_csp_cbf/ec/bite"
DEPLOYER_FQDN = "mid_csp_cbf/ec/deployer"

# Tango host environment variable
TANGO_HOST = "databaseds-tango-base." + namespace + ".svc.cluster.local:10000"
SKUID_POD = skuid_id + "." + namespace + ".svc.cluster.local:5004"

# For now, use direct cluster IP/port access for kafka and sku id pods
SKUID_POD = "10.101.212.237:9870"
KAFKA_POD = "10.110.145.7:9092"

# Parent directory to use to grab config files.
DATA_DIR = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "data")
# Config file directories
COMMON_CONFIG = os.path.join(DATA_DIR, "mid_telescope/cbf")
CSP_CONFIG = os.path.join(DATA_DIR, "mid_telescope/csp")
TMC_CONFIG = os.path.join(DATA_DIR, "mid_telescope/tmc")
HW_CONFIG = os.path.join(DATA_DIR, "psi/cbf/hw_config")
SLIM_CONFIG = os.path.join(COMMON_CONFIG, "slim_config")
CBF_INPUT_DIR = os.path.join(COMMON_CONFIG, "cbf_input_data")
# For mapping the talon boards to receptor
RECEPTOR_MAP = ["SKA001", "SKA036", "SKA063", "SKA100"]

In [ ]:
print(HW_CONFIG)

Next, set the environment arg for TANGO HOST, SKUID_POD and KAFKA, along with the oda API service endpoint:

In [ ]:
os.environ["TANGO_HOST"] = TANGO_HOST
os.environ["SKUID_URL"] = SKUID_POD
os.environ["ODA_URL"] = f"http://142.73.34.170/{namespace}/oda/api/v5/"
print("SKUID service set to: ", os.environ["SKUID_URL"])
print("TANGO_HOST set to:", os.environ["TANGO_HOST"])
print("ODA endpoint set to:{}".format(os.environ["ODA_URL"]))
print("This can be checked via {}ui".format(os.environ["ODA_URL"]))

In [ ]:
os.environ["telescope"] = "mid"

With all the file paths defined the JSON files can be loaded in and checked.

In [ ]:
print("Getting files...")

INIT_SYS_PARAM_FILE = os.path.join(COMMON_CONFIG, "sys_params/initial_system_param.json")
ASSIGN_RESOURCES_FILE = os.path.join(TMC_CONFIG, "assign_resources.json")
CONFIGURE_SCAN_FILE = os.path.join(TMC_CONFIG, "configure_scan.json")
SCAN_FILE = os.path.join(TMC_CONFIG, "scan.json")

CBF_INPUT_FILE = f"{CBF_INPUT_DIR}/cbf_input_data.json"
BITE_CONFIG_FILE = f"{CBF_INPUT_DIR}/bite_config_parameters/bite_configs.json"
FILTERS_FILE = f"{CBF_INPUT_DIR}/bite_config_parameters/filters.json"

DISH_CONFIG_FILE = f"{COMMON_CONFIG}/sys_params/load_dish_config.json"

START_CHANNEL = 0
END_CHANNEL = 14860
START_PORT = 21000

SCAN_COMBOS = [[0.0, 5.0], [0.0, -5.0], [5.0, 0.0], [-5.0, 0.0]]

files = [
    INIT_SYS_PARAM_FILE,
    ASSIGN_RESOURCES_FILE,
    CONFIGURE_SCAN_FILE,
    SCAN_FILE,
    CBF_INPUT_FILE,
    BITE_CONFIG_FILE,
    FILTERS_FILE,
    DISH_CONFIG_FILE,
]

for file in files:
    if os.path.isfile(file):
        print(f"{file} exists: ✔️")
    else:
        print(f"{file} does not exist ❌")

Next, set up the hw config to match the boards in use:

In [ ]:
if any(i > 4 for i in target_boards_list):
    print("Using swap for higher number talons")
    config = "hw_config_swap_psi.yaml"
    print("Modifying target to use lower nums to match swap file")
    target_boards_list = list(map(lambda x: x - 4, target_boards_list))

else:
    print("Using standard HW config")
    config = "hw_config_psi.yaml"

HW_CONFIG_FILE = os.path.join(HW_CONFIG, config)
if os.path.isfile(HW_CONFIG_FILE):
    print("HW config: ✔️")
else:
    print("hw config: ❌")

receptor_ids = list(map(lambda x: RECEPTOR_MAP[x - 1], target_boards_list))

If the HW config file exists, it is loaded into the pod:

In [ ]:
!kubectl cp $HW_CONFIG_FILE $namespace/ds-cbfcontroller-controller-0:/app/mnt/hw_config/hw_config.yaml 

### 1.3 Create Device Proxies

Using the FQDNs set earlier, and with the pod spun up, create device proxies to the devices used and check the connection to them.

In [ ]:
# CSP Devices
csp_controller = DeviceProxy(CSP_CONTROLLER_FQDN)
print(f"CSP Controller: {csp_controller.Status()}")
csp_subarray = DeviceProxy(CSP_SUBARRAY_FQDN)
print(f"CSP Subarray: {csp_subarray.Status()}")

# TMC Devices
tmc_central_node = DeviceProxy(CENTRAL_NODE_FQDN)
print(f"Central Node: {tmc_central_node.Status()}")
leaf_node_subarray = DeviceProxy(LEAF_NODE_SUBARRAY_FQDN)
print(f"Leaf Subarray Node: {leaf_node_subarray.Status()}")
tmc_subarray = DeviceProxy(TMC_SUBARRAY_FQDN)
print(f"TMC subarray Node: {tmc_subarray.Status()}")

leaf_node_master = DeviceProxy(LEAF_NODE_MASTER_FQDN)

# Deployer for setup and BITE for data mocking
bite = DeviceProxy(BITE_FQDN)
deployer = DeviceProxy(DEPLOYER_FQDN)

# CBF Device
cbf_subarray = DeviceProxy(CBF_SUBARRAY_FQDN)
print(f"CBF subarray: {cbf_subarray.Status()}")

### 1.4 Downloading Requirements via the Deployer

First, set the board to deploy to and turn on the deployer device.

In [ ]:
deployer.On()
deployer.targetTalons = target_boards_list
print(deployer.targetTalons)
deployer.generate_config_jsons()

Once started and configured, the required devices can then be downloaded.

In [ ]:
deployer.set_timeout_millis(400000)
try:
    deployer.download_artifacts()
except DevFailed as e:
    print(e)
    print(
        "Timed out, this is likely due to the download taking some time. Check the logs with the code space below after some time to see if it passes."
    )
deployer.set_timeout_millis(3000)

Now configure the device database with the downloaded devices:

In [ ]:
deployer.configure_db()

### 1.5 Set up Execution Block

The first step is to generate an empty Execution Block, using the help to work with the API endpoint:

In [ ]:
exec_block_id = oda_helper.create_eb(telescope="ska_mid")
print(exec_block_id)

## 2 Preparing Telescope

Start by setting the admin modes to allow for control of the telescope:

In [ ]:
csp_controller.write_attribute("adminMode", 0)
tmc_central_node.write_attribute("adminMode", 0)
# Set CBF Simulation mode to false and CBF timeout to 99s
csp_controller.cbfSimulationMode = 0
csp_controller.commandTimeout = 99

Start by loading in the VCC config (this step may need to be re-run):

In [ ]:
with open(DISH_CONFIG_FILE, encoding="utf-8") as f:
    dish_config_json = json.load(f)

dish_config_json["tm_data_sources"][
    0
] = "car://gitlab.com/ska-telescope/ska-telmodel-data?0.1.0-rc-mid-itf#tmdata"
dish_config_json["tm_data_filepath"] = "instrument/ska1_mid_itf/ska-mid-cbf-system-parameters.json"

print(f"dish_config_json file contents: \n{dish_config_json}")
tmc_central_node.LoadDishCfg(json.dumps(dish_config_json))

sleep(2)
print(f"TMC CSP Master's Dish Vcc Config attribute value: \n{leaf_node_master.dishVccConfig}")
print(
    f"\nTMC CSP Master's Source Dish Vcc Config attribute value: \n{leaf_node_master.sourceDishVccConfig}"
)

Before continuing, ensure this

In [ ]:
if tmc_central_node.isDishVccConfigSet:
    print("DishVCC set, can continue!")
else:
    print("DishVCC not set, repeat last step.")

Set up the objects for controlling the telescope via OSO:

In [ ]:
sub = SubArray(subarray_id)
tel = Telescope()

Now, while monitoring the Taranta dashboard, send the ON command to the telescope and monitor that the LURs Controller and subarrays come on:

In [ ]:
tel.on()

### 2.1 Set up CSP and TMC

In [ ]:
print(f"CSP controller state is: {csp_controller.state()}")
print(f"Central node state is: {tmc_central_node.state()}")

### 2.2 Define Scheduling Block

Temporary work around for ingress issues:

In [ ]:
os.environ["SKUID_URL"] = SKUID_POD
print("SKUID service set to: ", os.environ["SKUID_URL"])

Now, use the SKUID service to generate processing and execution block IDs from the scheduling block definition.

In [ ]:
sb = load_sbd("./sb/mid_5_point_sb.json")
sbi = create_sbi(sb)

sbi.sdp_configuration.execution_block.eb_id = exec_block_id

In [ ]:
sb_json = SBDefinition.model_dump_json(sb)
sb_json = sb_json.replace("<KAFKA_HOST>", KAFKA_POD)
sb = SBDefinition.model_validate_json(sb_json)